*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

## Timing and Profiling

In the process of developing code, there are often trade-offs you can make between various implementations.
Early in developing your algorithm, it can be counterproductive to worry about such things, and we have not worried about it until now. As Donald Knuth famously quipped, "We should forget about small efficiencies, say about 97% of the time: premature optimization is the root of all evil."

But once you have your code working, it can be useful to dig into its efficiency a bit.
Sometimes it's useful to check the execution time of a given command or set of commands; other times it's useful to dig into a multiline process and determine where the bottleneck lies in some complicated series of operations.
IPython provides access to a wide array of functionality for this kind of timing and profiling of code.
Here we'll discuss the following IPython magic commands:

- ``%time``: Time the execution of a single statement
- ``%timeit``: Time repeated execution of a single statement for more accuracy
- ``%prun``: Run code with the profiler
- ``%lprun``: Run code with the line-by-line profiler
- ``%memit``: Measure the memory use of a single statement
- ``%mprun``: Run code with the line-by-line memory profiler

The last four commands are not bundled with IPython by default – you may need to install the ``line_profiler`` and ``memory_profiler`` modules to use these on your own installation. Note that these magic functions using the `%` symbol are available to us here in Jupyter because they are provided by IPython, but these will not work in regular Python environments. There are alternative ways to use these modules without using magic functions, but we will not cover those methods here.

If you are using Anaconda on your own machine and need to install these packages it can be easily done from the Anaconda Navigator. If you open the Navigator, and click on *Environments* in the left hand menu you should be presented with a list of packages already installed. If you change the dropdown at the top to *Not Installed* and then search for any of the packages below you will be able to select the package you wish to install and hit the *Apply* button in the bottom right of the Navigator.

### Timing Code Snippets: ``%timeit`` and ``%time``

We can use the ``%timeit`` line-magic and ``%%timeit`` cell-magic to time the repeated execution of snippets of code. We can place the `%timeit` command in front of any line of Python code to time how long it takes to run just that single line. We use the `%%timeit` command to time all of the code within a particular Jupyter cell. The code below may take a little while to run. So long as you can see the `In [*]` on the left hand side it should be doing something useful.

In [ ]:
%timeit sum(range(100))

Note that because this operation is so fast, ``%timeit`` automatically does a large number of repetitions. In this case you will likely get a result that looks like: `1.36 µs ± 1.18 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)`

This might look a bit strange, so lets break it down. Apparently we did 7 runs, so we repeated the same test 7 times to calculate an average. Each of those runs consisted of 1000000 loops, so we repeated this line of code 1000000 times in each of our 7 runs. The runtime of 1.36 microseconds is the average runtime of those 7 runs divided by 1000000, meaning each run took about 1.36 seconds, and our 7 runs take around 10 seconds. If you compare this to your stopwatch you will see a similar figure.


For slower commands, ``%timeit`` will automatically adjust and perform fewer repetitions. This time we will measure the time taken to run the entire cell:

In [ ]:
%%timeit
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

We still likely did 7 runs in order to calculate an average, but on each of those runs we only ran the cell once, we didn't run it multiple times as in the previous example.

Sometimes repeating an operation is not the best option.
For example, if we have a list that we'd like to sort, we might be misled by a repeated operation.
Sorting a pre-sorted list is much faster than sorting an unsorted list, so the repetition will skew the result:

In [ ]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

For this, the ``%time`` magic function may be a better choice. It also is a good choice for longer-running commands, when short, system-related delays are unlikely to affect the result.
Let's time the sorting of an unsorted and a presorted list:

In [ ]:
import random
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()

In [ ]:
print("sorting an already sorted list:")
%time L.sort()

Notice how much faster the presorted list is to sort, but notice also how much longer the timing takes with ``%time`` versus ``%timeit``, even for the presorted list!
This is a result of the fact that ``%timeit`` does some clever things under the hood to prevent system calls from interfering with the timing.
For example, it prevents cleanup of unused Python objects (known as *garbage collection*) which might otherwise affect the timing.
For this reason, ``%timeit`` results are usually noticeably faster than ``%time`` results.

For ``%time`` as with ``%timeit``, using the double-percent-sign cell magic syntax allows timing of multiline scripts:

In [ ]:
%%time
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

For more information on ``%time`` and ``%timeit``, as well as their available options, use the IPython help functionality (i.e., type ``%time?`` in a new cell or at the IPython prompt).

## Profiling Full Scripts: ``%prun``

A program is made of many single statements, and sometimes timing these statements in context is more important than timing them on their own.
Python contains a built-in code profiler (which you can read about in the Python documentation), but IPython offers a much more convenient way to use this profiler, in the form of the magic function ``%prun``.

By way of example, we'll define a simple function that does some calculations:

In [ ]:
def sum_of_lists(N):
    total = 0
    for i in range(5000):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

Now we can call ``%prun`` with a function call to see the profiled results. We also pass the parameter `-l 5` to limit the output to the 5 most time consuming calls.

In [ ]:
%prun -l 5 sum_of_lists(10000)

In the notebook, the output is printed to the pager, and looks something like this:

```
14 function calls in 0.714 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.599    0.120    0.599    0.120 <ipython-input-19>:4(<listcomp>)
        5    0.064    0.013    0.064    0.013 {built-in method sum}
        1    0.036    0.036    0.699    0.699 <ipython-input-19>:1(sum_of_lists)
        1    0.014    0.014    0.714    0.714 <string>:1(<module>)
        1    0.000    0.000    0.714    0.714 {built-in method exec}
```

The result is a table that indicates, in order of total time on each function call, where the execution is spending the most time. In my case, the bulk of execution time is in the list comprehension inside ``sum_of_lists``. On NCC you might see time allocated to some other functions related to Jupyterhub internal communication if the system is busy. 
From here, we could start thinking about what changes we might make to improve the performance in the algorithm.

For more information on ``%prun``, as well as its available options, use the IPython help functionality (i.e., type ``%prun?``  in a new cell or at the IPython prompt).

## Line-By-Line Profiling with ``%lprun``

The function-by-function profiling of ``%prun`` is useful, but generally it's more convenient to have a line-by-line profile report.
This is not built into Python or IPython, but there is a ``line_profiler`` package available for installation that can do this.
If you are using your own Python install you may have to do a ``pip`` install:

```
$ pip install line_profiler
```

You can optionally do this in your notebook with the code block below. Note that you **DO NOT** need to run this if you are using NCC.

In [ ]:
!pip install line_profiler

Next, you can use IPython to load the ``line_profiler`` IPython extension, offered as part of this package:

In [ ]:
%load_ext line_profiler

Now the ``%lprun`` command will do a line-by-line profiling of any function–in this case, we need to tell it explicitly which functions we're interested in profiling:

In [ ]:
%lprun -f sum_of_lists sum_of_lists(5)

As before, the notebook sends the result to the pager, but it looks something like this:

```
Timer unit: 1e-09 s

Total time: 0.0246005 s
File: /tmp/ipykernel_3530330/1228561257.py
Function: sum_of_lists at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def sum_of_lists(N):
     2         1       2236.0   2236.0      0.0      total = 0
     3      5001    2384255.0    476.8      9.7      for i in range(5000):
     4     30000   19043188.0    634.8     77.4          L = [j ^ (j >> i) for j in range(N)]
     5      5000    3169218.0    633.8     12.9          total += sum(L)
     6         1       1597.0   1597.0      0.0      return total
```

The information at the top gives us the key to reading the results: the time is reported in microseconds and we can see where the program is spending the most time.
At this point, we may be able to use this information to modify aspects of the script and make it perform better for our desired use case.

For more information on ``%lprun``, as well as its available options, use the IPython help functionality (i.e., type ``%lprun?`` in a new cell or at the IPython prompt).

## Profiling Memory Use: ``%memit`` and ``%mprun``

Another aspect of profiling is the amount of memory an operation uses.
This can be evaluated with another IPython extension, the ``memory_profiler``.
As with the ``line_profiler``, we start by ``pip``-installing the extension:

```
$ pip install memory_profiler
```
You can optionally do this in your notebook with the code block below. Note that you **DO NOT** need to run this if you are using NCC.

In [ ]:
!pip install memory_profiler

Then we can use IPython to load the extension:

In [ ]:
%load_ext memory_profiler

The memory profiler extension contains two useful magic functions: the ``%memit`` magic (which offers a memory-measuring equivalent of ``%timeit``) and the ``%mprun`` function (which offers a memory-measuring equivalent of ``%lprun``).
The ``%memit`` function can be used rather simply:

In [ ]:
%memit sum_of_lists(1000)

We see that this function uses about 65 MB of memory. (You might want to restart your kernel first to get a number this low, otherwise it might be closer to 150MB if you've run this whole notebook. The profiler appears to include memory usage of everything run thus far in the notebook which isn't ideal!)

For a line-by-line description of memory use, we can use the ``%mprun`` magic. This works slightly different, and requires that our function be defined outside of our notebook. If yuo want to see sensible results I also recommended that you restart the kernel first.
We can use the `%%file` magic command to write our function out to a file and then make use of the memory profiler as follows: 

In [ ]:
%%file mprun_demo.py
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

In [ ]:
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(20000)

You might notice some unusual results with massive negative numbers, particularly if you increase the parameter. It appears that there are still some issues with the `%mprun` magic function, as has been the case in the last few versions! If all were working well you would expect to see something more along the lines of:

```
Filename: ./mprun_demo.py

Line #    Mem usage    Increment   Line Contents
================================================
     1     39.0 MiB      0.0 MiB   def sum_of_lists(N):
     2     39.0 MiB      0.0 MiB       total = 0
     3     46.5 MiB      7.5 MiB       for i in range(5):
     4     71.9 MiB     25.4 MiB           L = [j ^ (j >> i) for j in range(N)]
     5     71.9 MiB      0.0 MiB           total += sum(L)
     6     46.5 MiB    -25.4 MiB           del L # remove reference to L
     7     39.1 MiB     -7.4 MiB       return total

```